In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import onnx
from onnx_tf.backend import prepare

ModuleNotFoundError: No module named 'onnx'

In [2]:
class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} \tLoss: {:.6f}'.format(
                    epoch,  loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the maxlog-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
        

In [4]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, 
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=1000, shuffle=True)

In [5]:
device = torch.device("cuda")
print(device)
model_torch = Net().to(device)
optimizer = optim.SGD(model_torch.parameters(), lr=0.01, momentum=0.5)
for epoch in range(10):
    train(model_torch, device, train_loader, optimizer, epoch)
    test(model_torch, device, test_loader) 

cuda
Train Epoch: 0 	Loss: 2.324259

Test set: Average loss: 0.1785, Accuracy: 9446/10000 (94%)

Train Epoch: 1 	Loss: 0.256421

Test set: Average loss: 0.1137, Accuracy: 9660/10000 (97%)

Train Epoch: 2 	Loss: 0.271181

Test set: Average loss: 0.0918, Accuracy: 9715/10000 (97%)

Train Epoch: 3 	Loss: 0.208402

Test set: Average loss: 0.0773, Accuracy: 9751/10000 (98%)

Train Epoch: 4 	Loss: 0.334902

Test set: Average loss: 0.0697, Accuracy: 9781/10000 (98%)

Train Epoch: 5 	Loss: 0.089708

Test set: Average loss: 0.0661, Accuracy: 9770/10000 (98%)

Train Epoch: 6 	Loss: 0.141611

Test set: Average loss: 0.0611, Accuracy: 9801/10000 (98%)

Train Epoch: 7 	Loss: 0.170826

Test set: Average loss: 0.0568, Accuracy: 9823/10000 (98%)

Train Epoch: 8 	Loss: 0.139686

Test set: Average loss: 0.0549, Accuracy: 9819/10000 (98%)

Train Epoch: 9 	Loss: 0.079753

Test set: Average loss: 0.0561, Accuracy: 9833/10000 (98%)



In [6]:
torch.save(model_torch.state_dict(), 'mnist.pth')

In [67]:
model_torch = Net()
model_torch.load_state_dict(torch.load('mnist.pth'))
dummy_input = Variable(torch.randn(1, 1, 28, 28))
torch.onnx.export(model_torch.eval(), dummy_input, "mnist.onnx")

In [68]:
model_onnx = onnx.load('mnist.onnx')
model_tf = prepare(model_onnx)

In [74]:
import numpy as np
from IPython.display import display
from PIL import Image
import matplotlib.pyplot as plt

print('Image 1:')
img = Image.open('two.png').resize((28, 28)).convert('L')
display(img)
temp = np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :]
print(temp.shape)
output = model_torch(torch.from_numpy(temp))
print('The digit is classified as', output.detach().numpy(), 'pytorch')
output = model_tf.run(temp)
print('The digit is classified as', output, 'tf')

Image 1:


(1, 1, 28, 28)
The digit is classified as [[-1172.1316 -1628.2738 -1390.8823 -2273.5647 -1083.5457 -1100.4241
      0.     -2661.3796 -1179.9258 -1963.0388]] pytorch
The digit is classified as Outputs(_0=array([[-1172.1315, -1628.2738, -1390.8823, -2273.5647, -1083.5457,
        -1100.424 ,     0.    , -2661.3796, -1179.9258, -1963.0388]],
      dtype=float32)) tf


In [76]:
print('Image 2:')
img = Image.open('three.png').resize((28, 28)).convert('L')
display(img)
temp = np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :]
print(temp.shape)
output = model_torch(torch.from_numpy(temp))
print('The digit is classified as', output.detach().numpy(), 'pytorch')
output = model_tf.run(temp)
print('The digit is classified as', output, 'tf')

Image 2:


(1, 1, 28, 28)
The digit is classified as [[-1828.054  -1347.0144 -1056.4312     0.     -1870.2683 -1012.4519
  -2525.1013 -1003.2545 -1007.5098  -741.2179]] pytorch
The digit is classified as Outputs(_0=array([[-1828.0537 , -1347.0143 , -1056.4309 ,     0.     , -1870.2682 ,
        -1012.452  , -2525.101  , -1003.25433, -1007.50964,  -741.2178 ]],
      dtype=float32)) tf


In [77]:
model_tf.export_graph('mnist.pb')

INFO:tensorflow:Assets written to: mnist.pb\assets


INFO:tensorflow:Assets written to: mnist.pb\assets


In [144]:
for param in model_torch.parameters():
    print(param.data)
    print(param.data.shape)

tensor([[[[ 2.3621e-01,  1.3497e-01, -1.2790e-01, -1.0865e-01,  9.7583e-02],
          [ 2.9348e-01,  8.4625e-02, -3.5057e-03, -2.9974e-01,  1.1580e-01],
          [-8.7698e-03,  1.2850e-02, -1.6791e-01, -1.4662e-01, -1.2763e-01],
          [-4.4721e-02, -1.6092e-01,  1.3615e-01,  2.2192e-01,  2.4346e-02],
          [ 1.4238e-01,  1.7062e-01,  2.7300e-01, -1.3579e-01, -1.1499e-01]]],


        [[[ 2.4378e-01,  3.6939e-01,  7.3117e-02,  1.8251e-01, -1.6565e-01],
          [ 9.3901e-03, -5.7588e-02, -3.1732e-01, -2.9014e-01, -3.0704e-01],
          [-2.3215e-01, -4.2892e-01, -3.7199e-01, -2.3679e-01,  5.1020e-02],
          [-2.4692e-01, -2.4577e-01,  4.8212e-02,  3.3047e-02,  3.1894e-01],
          [ 2.0490e-01,  1.9171e-01,  3.3009e-01,  2.9320e-01,  2.3797e-01]]],


        [[[ 2.5635e-01,  2.2497e-01,  1.0351e-01, -1.3179e-01, -1.0490e-01],
          [-1.1305e-01,  2.5210e-01,  2.5358e-01,  5.9893e-02, -2.4075e-01],
          [ 3.3765e-02,  1.6204e-01,  5.0856e-02,  5.0694e-02, -2.69

In [102]:
import tensorflow as tf
imported = tf.saved_model.load('mnist.pb')
f = imported.signatures["serving_default"]
#print(f(x=tf.constant([[1.]])))

In [108]:
f(tf.convert_to_tensor(temp))

{'20': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[-1828.0537 , -1347.0143 , -1056.4309 ,     0.     , -1870.2682 ,
         -1012.452  , -2525.101  , -1003.25433, -1007.50964,  -741.2178 ]],
       dtype=float32)>}

In [114]:
model_new = tf.keras.models.load_model('mnist.pb')
f_new = model_new.signatures["serving_default"]

In [143]:
f_new(tf.convert_to_tensor(temp))

{'20': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[-1828.0537 , -1347.0143 , -1056.4309 ,     0.     , -1870.2682 ,
         -1012.452  , -2525.101  , -1003.25433, -1007.50964,  -741.2178 ]],
       dtype=float32)>}

In [141]:
print(type(f_new.inputs[0]))

AttributeError: 'Tensor' object has no attribute 'show'

In [7]:
import torch
from torch import nn
cuda = torch.device('cuda')
print(cuda)

cuda


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(cuda)
print(model)

RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination

In [5]:
x = torch.randn(1, 10)

class MyModel(nn.Module):
    def __init__(self, inplace=False):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 10)
        self.fc1 = nn.Linear(10, 1)
        self.relu = nn.ReLU(inplace=inplace)
        
    def forward(self, x):
        x = self.relu(self.fc1(self.fc(x)))
        return x

model = MyModel()

In [6]:


model.eval()
# out of place
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
d = dict(model.named_modules())

for i in d:
    print(i)
    model.fc1.register_forward_hook(get_activation(i))

model.fc.register_forward_hook(get_activation('fc'))
output = model(x)
activation['fc']


fc
fc1
relu


tensor([[-1.0813]])

In [12]:
model.eval()
hooks = {}
def get_activation(name):
    def hook(model, input, output):
        hooks[name] = output.detach()
    return hook

for name, module in model.named_modules():
    print(name)
    hooks[name] = module.register_forward_hook(get_activation(name))
    #output = model(x)
    print(hooks[name])
lists = model.named_modules()
print(lists[2][0])



fc
fc1
relu


TypeError: 'generator' object is not subscriptable

In [9]:
output = model(x)
print(hooks['fc'])
output = model(x)
print(hooks['fc1'])

tensor([[-0.3853,  0.8042,  1.2120, -0.5853,  0.2316, -1.4203, -0.0124,  1.0610,
          0.2698, -1.4225]])
tensor([[-1.0813]])
